In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy

Using TensorFlow backend.


In [2]:
path = '../data/nparrs/'
savepath = '../data/'
images = glob.glob(path+'*.npy')
totalImages = 1800

In [3]:
def myGenerator(batch_size):
    while True:
        index_list = random.sample(range(1, totalImages), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = images[i]
            frame = np.load(frame)
            tile_index = np.random.randint(0, 199)
            #print(i, tile_index, frame.shape)
            alldata_x.append(tile_index*totalImages+i)
            alldata_y.append(frame[tile_index])
        alldata_x = np.array(alldata_x)
        #alldata_x = np.rollaxis(alldata_x, 1, 5)  
        #alldata_x = alldata_x.reshape((32, 30, 240, 480, 3))
        #alldata_x = np.swapaxes(alldata_x, 1, 4)
        alldata_y = np.array(alldata_y)
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        yield alldata_x, alldata_y
#x = myGenerator()
#xtrain, ytrain = next(x)
#print('xtrain shape:',xtrain.shape)
#print('ytrain shape:',ytrain.shape)

In [4]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 192
        self.img_cols = 192
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 360000
        self.latent_dim  = 200
        self.embedding_layer = Embedding(self.num_classes, self.latent_dim)
        
        self.test_tiles = np.array([np.random.randint(0, 1800*200, 10)])
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss=['mse'],
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise and the target label as input
        # and generates the corresponding digit of that label
        noise = Input(shape=(1,))
        #label = Input(shape=(1,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        valid = self.discriminator([img, noise])

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['mse'],
            optimizer=optimizer)

    def build_generator(self):
        model = Sequential()
        model.add(Dense(192 * 192 * 3, activation='relu', input_dim=self.latent_dim))
        model.add(Reshape((192, 192, 3)))
        model.add(Conv2D(256, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        label           = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(self.embedding_layer(label))
        img             = model(label_embedding)

        model2 =  Model(label, img)
        print(model2.summary())
        return model2

    def build_discriminator(self):
        img   = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')
        label_embedding = Flatten()(self.embedding_layer(label))
        
        label_dense = Dense(100)(label_embedding)
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=(192, 192, 3)))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64,  (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (6, 6),  strides=(2, 2)))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(128, (3, 3)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Conv2D(128, (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        #model.add(Dense(512))
        #model.add(LeakyReLU(alpha=0.2))
        
        dense100    = model(img)
        conditioned = multiply([dense100, label_dense])
        #dense64     = Dense(128)(conditioned)
        #dense64l    = LeakyReLU(alpha=0.2)(dense64)
        dense32     = Dense(32)(conditioned)
        dense32l    = LeakyReLU(alpha=0.2)(dense32)
        
        #For ls gan
        validity = Dense(1)(dense32l)
        #validity = Dense(1, activation='sigmoid')(dense32l)
        
        model3 = Model([img, label], validity) 
        print(model3.summary())
        return model3

    def train(self, epochs, batch_size=128, sample_interval=50):
        random.seed(10)
        
        # Load the dataset
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            
            # Adversarial ground truths
            valid = np.ones((batch_size, 1))
            fake  = np.zeros((batch_size, 1))
            
            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            #idx = np.random.randint(0, X_train.shape[0], batch_size)
            noise, imgs = X_train, y_train

            # Sample noise as generator input
            # noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch([imgs, noise], valid)
            d_loss_fake = self.discriminator.train_on_batch([gen_imgs, noise], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Condition on labels
            #sampled_labels = np.random.randint(0, 7, batch_size).reshape(-1, 1)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                #self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')
                self.discriminator.save_weights(savepath+'weights/discriminator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c = 1, 7
        noise          = self.test_tiles

        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        temp = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        #print(gen_imgs[0].shape)
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4], gen_imgs[5], gen_imgs[6]])
        #print(combined.shape)
        combined = np.hstack(combined.reshape(7,192,192, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)

In [5]:
cgan = CGAN()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 200)       72000000    input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 192, 192, 3)  0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 200)          0           embedding_1[0][0]                
__________________________________________________________________________________________________
sequential

In [ ]:
cgan.train(1000000, batch_size=10, sample_interval=1000)

0 [D loss: 0.426035, acc.: 50.00%] [G loss: 0.736480]
1 [D loss: 0.454116, acc.: 50.00%] [G loss: 0.682353]
2 [D loss: 0.439750, acc.: 50.00%] [G loss: 0.646172]
3 [D loss: 0.398924, acc.: 50.00%] [G loss: 0.687595]
4 [D loss: 0.369145, acc.: 55.00%] [G loss: 0.516780]
5 [D loss: 0.396341, acc.: 50.00%] [G loss: 0.557979]
6 [D loss: 0.416612, acc.: 50.00%] [G loss: 0.593036]
7 [D loss: 0.371029, acc.: 50.00%] [G loss: 0.527829]
8 [D loss: 0.339106, acc.: 50.00%] [G loss: 0.557505]
9 [D loss: 0.341632, acc.: 50.00%] [G loss: 0.445864]
10 [D loss: 0.354653, acc.: 55.00%] [G loss: 0.489229]
11 [D loss: 0.328518, acc.: 50.00%] [G loss: 0.376544]
12 [D loss: 0.307437, acc.: 55.00%] [G loss: 0.390481]
13 [D loss: 0.377286, acc.: 35.00%] [G loss: 0.317583]
14 [D loss: 0.366653, acc.: 45.00%] [G loss: 0.395787]
15 [D loss: 0.340335, acc.: 50.00%] [G loss: 0.377513]
16 [D loss: 0.347185, acc.: 50.00%] [G loss: 0.351297]
17 [D loss: 0.268869, acc.: 65.00%] [G loss: 0.343631]
18 [D loss: 0.266360

149 [D loss: 0.119631, acc.: 80.00%] [G loss: 0.816122]
150 [D loss: 0.120417, acc.: 90.00%] [G loss: 0.805658]
151 [D loss: 0.155687, acc.: 75.00%] [G loss: 0.567500]
152 [D loss: 0.164161, acc.: 75.00%] [G loss: 0.757974]
153 [D loss: 0.183110, acc.: 80.00%] [G loss: 0.976950]
154 [D loss: 0.050877, acc.: 90.00%] [G loss: 0.909090]
155 [D loss: 0.091996, acc.: 90.00%] [G loss: 0.843868]
156 [D loss: 0.118960, acc.: 85.00%] [G loss: 0.883803]
157 [D loss: 0.134374, acc.: 80.00%] [G loss: 0.849309]
158 [D loss: 0.085463, acc.: 90.00%] [G loss: 0.787465]
159 [D loss: 0.063984, acc.: 95.00%] [G loss: 0.783770]
160 [D loss: 0.054994, acc.: 100.00%] [G loss: 0.764833]
161 [D loss: 0.120183, acc.: 90.00%] [G loss: 0.773226]
162 [D loss: 0.083108, acc.: 90.00%] [G loss: 0.731245]
163 [D loss: 0.053622, acc.: 100.00%] [G loss: 0.789911]
164 [D loss: 0.093464, acc.: 95.00%] [G loss: 0.834433]
165 [D loss: 0.186985, acc.: 75.00%] [G loss: 0.744236]
166 [D loss: 0.087084, acc.: 90.00%] [G loss: 